<a href="https://colab.research.google.com/github/victorog17/soulcode_aulas_apache_beam/blob/main/Apache_Beam_Salvando_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALANDO APACHE BEAM GCP**

In [ ]:
pip install apache-beam[gcp]

**BIBLIOTECAS**

In [1]:
import apache_beam as beam
import os

**SALVANDO EM CLOUD**

In [2]:
serviceAccount = '/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/ingestao-apache-beam-victor-0ac6fa7d11ea.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

p1 = beam.Pipeline()

atrasados_tempo = (
    p1
    |'1.Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'1.Separador' >> beam.Map(lambda record: record.split(','))
    |'1.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'1.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'1.Combinar os dados' >> beam.CombinePerKey(sum)
    #|'1.Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

atrasos_qtd = (
    p1
    |'2.Extrair os dados' >> beam.io.ReadFromText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/voos_sample.csv', skip_header_lines= 0)
    |'2.Separador' >> beam.Map(lambda record: record.split(','))
    |'2.Aeroportos atrasados' >> beam.Filter(lambda record: int(record[8]) > 0)
    |'2.Agregação de colunas' >> beam.Map(lambda record: (record[3], int(record[8])))
    |'2.Quantidade de atrasos' >> beam.combiners.Count.PerKey()
    #|'2.Gravar resultado' >> beam.io.WriteToText('/content/drive/MyDrive/Soul_Code_Academy/repositorio_apache_beam/resultato_voos_atrasados.txt')
)

tabela_final = (
    {'3.Qtd_Atrasos': atrasos_qtd, 'Tempo_Atrasos': atrasados_tempo}
    |'3.Agrupamento das Pipes' >> beam.CoGroupByKey()
    #|'3.Imprimir resultado' >> beam.Map(print)
    |'3.Salvando no Storage' >> beam.io.WriteToText('gs://eng_gcp_dados_victor_soulcode001/TESTE2.csv')
)
p1.run()